In [ ]:
!pip install networkx~=2.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


In [ ]:
import pickle
import copy
import warnings
warnings.filterwarnings('ignore') # It allows you to ignore warnings from specified module

import pandas as pd
import numpy as np
import networkx as nx

In [ ]:
fp = open('./data/twitch_missing_edges_final.p', "rb")
missing_edges = pickle.load(fp)
type(missing_edges)

set

In [ ]:
#df_neg - missing edges
#df_pos - present edges

df_neg = pd.DataFrame(list(missing_edges), columns=['Source', 'Destination']) # stores as a table format
print (df_neg.shape)
df_neg.head(1)

(35324, 2)


,Source,Destination
0,10847,50740


In [ ]:
df_pos = pd.read_csv('./data/musae_ENGB_edges.csv')
df_pos = df_pos.rename(columns = {'from':'Source', 'to':'Destination'})
df_pos = df_pos.drop_duplicates()
print (df_pos.shape)
print (type(df_pos))
df_pos.head(1)

(35324, 2)
<class 'pandas.core.frame.DataFrame'>


,Source,Destination
0,6194,255


In [ ]:
df_pos['Class'] = 1
df_pos.head(5)

,Source,Destination,Class
0,6194,255,1
1,6194,980,1
2,6194,2992,1
3,6194,2507,1
4,6194,986,1


In [ ]:
df_neg['Class'] = 0
df_neg.head(5)

,Source,Destination,Class
0,10847,50740,0
1,49613,19406,0
2,79713,11464,0
3,36081,36754,0
4,17670,39939,0


In [ ]:
frames = [df_pos, df_neg]
df = pd.concat(frames)
print(df.shape)
df.head(5)

(70648, 3)


,Source,Destination,Class
0,6194,255,1
1,6194,980,1
2,6194,2992,1
3,6194,2507,1
4,6194,986,1


In [ ]:
df.to_csv('./data/twitch_subset.csv')

In [ ]:
df_copy = copy.deepcopy(df)

## Feature Extraction

In [ ]:
g = nx.from_pandas_edgelist(df[['Source','Destination']], source='Source', target='Destination',create_using=nx.DiGraph())
print(nx.info(g))

DiGraph with 50119 nodes and 70648 edges


In [ ]:
# PageRank computes a ranking of the nodes in the graph G based on the structure of the incoming links
pr = nx.pagerank(g)
df['Page_Rank_Src'] = df.Source.apply(lambda row: pr.get(row))
df['Page_Rank_Dst'] = df.Destination.apply(lambda row: pr.get(row))
print (df)

       Source  Destination  Class  Page_Rank_Src  Page_Rank_Dst
0        6194          255      1       0.000013       0.000021
1        6194          980      1       0.000013       0.000035
2        6194         2992      1       0.000013       0.000014
3        6194         2507      1       0.000013       0.000019
4        6194          986      1       0.000013       0.000014
...       ...          ...    ...            ...            ...
35319   15120        47969      0       0.000019       0.000027
35320   27161        51743      0       0.000010       0.000012
35321    1829        13701      0       0.000019       0.000021
35322    7168         8823      0       0.000010       0.000019
35323    3621        28125      0       0.000027       0.000032

[70648 rows x 5 columns]


In [ ]:
# Shortest Path
def get_shortest_path(x, y):
    d = -1
    try:
        if g.has_edge(x, y):
            g.remove_edge(x, y)
            d = nx.shortest_path_length(g, source=x, target=y)
            g.add_edge(x, y)
        else:
            d = nx.shortest_path_length(g, source=x, target=y)
        if d>6:
          d=-1
    except:
        d = -1
    return d

df['Shortest_Path'] = df.apply(lambda row: get_shortest_path(row['Source'], row['Destination']), axis = 1)

In [ ]:
# Follows Back
def get_follows_back(x, y):
    return 1 if g.has_edge(y, x) else 0

df['Follows_Back'] = df.apply(lambda row: get_follows_back(row['Source'], row['Destination']), axis = 1)

In [ ]:
# Follow Features

followers_src, followers_dst, followees_src, followees_dst, int_followers, int_followees = [], [], [], [], [], []
preattach_src,preattach_dst=[],[]
total_neighbour=[]

In [ ]:
# Predecessor and successor

def pre_suc(i,r):

    pre_src = set(g.predecessors(r['Source'])) if set(g.predecessors(r['Source'])) else set()
    suc_src = set(g.successors(r['Source'])) if set(g.successors(r['Source'])) else set()

    pre_dst = set(g.predecessors(r['Destination'])) if set(g.predecessors(r['Destination'])) else set()
    suc_dst = set(g.successors(r['Destination'])) if set(g.successors(r['Destination'])) else set()

    return pre_src,suc_src,pre_dst,suc_dst

In [ ]:
# Followers followees - source and destination

def follower_followee(pre_src_no,suc_src_no,pre_dst_no,suc_dst_no):

    followers_src.append(pre_src_no)
    followees_src.append(suc_src_no)

    followers_dst.append(pre_dst_no)
    followees_dst.append(suc_dst_no)


In [ ]:
# Common neighbours

def common_neighbour(pre_src,suc_src,pre_dst,suc_dst):

    int_followers.append(len(pre_src.intersection(pre_dst)))
    int_followees.append(len(suc_src.intersection(suc_dst)))


In [ ]:
# Preferential attachment

def preferential_attach(pre_src_no,suc_src_no,pre_dst_no,suc_dst_no):

    preattach_src.append( pre_src_no + suc_src_no )
    preattach_dst.append( pre_dst_no + suc_dst_no )


In [ ]:

for i, r in df.iterrows():

    pre_src,suc_src,pre_dst,suc_dst=pre_suc(i,r)

    pre_src_no=len(pre_src)
    suc_src_no=len(suc_src)
    pre_dst_no=len(pre_dst)
    suc_dst_no=len(suc_dst)

    follower_followee(pre_src_no,suc_src_no,pre_dst_no,suc_dst_no)

    common_neighbour(pre_src,suc_src,pre_dst,suc_dst)

    preferential_attach(pre_src_no,suc_src_no,pre_dst_no,suc_dst_no)

df['Followers_Src'] = followers_src
df['Followees_Src'] = followees_src
df['Followers_Dst'] = followers_dst
df['Followees_Dst'] = followees_dst
df['Int_Followers'] = int_followers
df['Int_Followees'] = int_followees
df['Preattach_Src'] = preattach_src
df['Preattach_Dst'] = preattach_dst



In [ ]:
print(df)

       Source  Destination  Class  Page_Rank_Src  Page_Rank_Dst  \
0        6194          255      1       0.000013       0.000021   
1        6194          980      1       0.000013       0.000035   
2        6194         2992      1       0.000013       0.000014   
3        6194         2507      1       0.000013       0.000019   
4        6194          986      1       0.000013       0.000014   
...       ...          ...    ...            ...            ...   
35319   15120        47969      0       0.000019       0.000027   
35320   27161        51743      0       0.000010       0.000012   
35321    1829        13701      0       0.000019       0.000021   
35322    7168         8823      0       0.000010       0.000019   
35323    3621        28125      0       0.000027       0.000032   

       Shortest_Path  Follows_Back  Followers_Src  Followees_Src  \
0                 -1             0              0              3   
1                 -1             0              0          

In [ ]:
df.to_csv('./data/twitch_final_dataset.csv')